# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [2]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [3]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [5]:
response = requests.post(
    "https://api.openai.com/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'chatcmpl-CqfHCoTAPH9badMzy7U2D4rYJjU1z',
 'object': 'chat.completion',
 'created': 1766667758,
 'model': 'gpt-5-nano-2025-08-07',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: Bananas are berries, but strawberries aren’t. Botanically, a berry comes from a single ovary with seeds inside; bananas fit that definition, while strawberries don’t. Want another fun fact?',
    'refusal': None,
    'annotations': []},
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 11,
  'completion_tokens': 820,
  'total_tokens': 831,
  'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
  'completion_tokens_details': {'reasoning_tokens': 768,
   'audio_tokens': 0,
   'accepted_prediction_tokens': 0,
   'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': None}

In [6]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: Bananas are berries, but strawberries aren’t. Botanically, a berry comes from a single ovary with seeds inside; bananas fit that definition, while strawberries don’t. Want another fun fact?'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [7]:
# Create OpenAI client

from openai import OpenAI
openai = OpenAI()

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: Honey never spoils. Archaeologists have found pots of honey in ancient Egyptian tombs that are still edible. Its longevity comes from low water content, acidity, and enzymes that bees add, which together create a hostile environment for microbes. Want another one on a different topic?'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [8]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



API key found and looks good so far!


In [9]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"Here's a fun one:\n\n**Octopuses have three hearts!**\n\nTwo of them pump blood to the gills, and the third pumps blood to the rest of the body. Pretty wild, right?"

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [10]:
requests.get("http://localhost:11434").content

b'Ollama is running'

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [12]:
!ollama pull llama3.2:1b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 246 KB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 1.6 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 3.0 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 3.7 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 5.1 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 6.4 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 7.1 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 8.5 MB/2.0 GB                  pulling manifest 
pulling dde5aa3f

In [16]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [17]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2:latest", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

'One fun fact is that there\'s a species of jellyfish that\'s immortal. The Turritopsis dohrnii, also known as the "immortal jellyfish," can transform its body into a younger state through a process called transdifferentiation. This means it can essentially revert back to its polyp stage and grow back into an adult again, making it theoretically biologically immortal!'

In [18]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏  89 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 767 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 2.2 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 3.5 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏ 4.2 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 5.6 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 7.0 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 7.7 MB/1.1 GB                  pulling manifest 
pulling aabd4deb

In [19]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

"What's the largest animal that can speak? 🐾"

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [ ]:
"""
Day 2 Homework: Website Summarizer with Ollama

Upgraded from Day 1 to use local Ollama models instead of OpenAI.
Class-based scraper with lazy evaluation, error handling, model comparison.
"""

# ============================================================================
# IMPORTS
# ============================================================================
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import os
from dotenv import load_dotenv

# ============================================================================
# CONFIGURATION
# ============================================================================

OLLAMA_BASE_URL = "http://localhost:11434/v1"
OLLAMA_MODEL = "llama3.2"  # Default model

# Model presets for quick switching
AVAILABLE_MODELS = {
    "fast": "llama3.2:1b",           # 1B params - fastest
    "balanced": "llama3.2",          # 3B params - default
    "powerful": "deepseek-r1:1.5b"   # Better reasoning
}

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')

# ============================================================================
# WEBSITE SCRAPER CLASS
# ============================================================================

class WebsiteScraper:
    """
    Class-based scraper with lazy evaluation.
    
    Parses HTML once in __init__, then provides title/content/links as properties.
    Uses @property decorators to compute only what's accessed.
    Caches results to avoid recomputation.
    
    Improvement over original scraper.py: avoids duplicate parsing.
    """
    
    def __init__(self, url):
        """Fetch and parse website. Raises ValueError on failure."""
        self.url = url
        
        try:
            self.response = requests.get(url, headers=headers, timeout=10)
            self.response.raise_for_status()
        except requests.Timeout:
            raise ValueError(f"Timeout fetching {url}")
        except requests.RequestException as e:
            raise ValueError(f"Failed to fetch {url}: {e}")
        
        self.soup = BeautifulSoup(self.response.content, "html.parser")
        self._title = None
        self._content = None
        self._links = None
    
    @property
    def title(self):
        """Extract title, cache on first access."""
        if self._title is None:
            self._title = self.soup.title.string if self.soup.title else "No title found"
        return self._title
    
    @property
    def content(self):
        """
        Extract cleaned text content.
        Removes script/style/img/input tags, truncates to 2000 chars.
        """
        if self._content is None:
            if self.soup.body:
                for irrelevant in self.soup.body(["script", "style", "img", "input"]):
                    irrelevant.decompose()
                text = self.soup.body.get_text(separator="\n", strip=True)
            else:
                text = ""
            self._content = (self.title + "\n\n" + text)[:2_000]
        return self._content
    
    @property
    def links(self):
        """Extract all links, filter out None/empty values."""
        if self._links is None:
            links = [link.get("href") for link in self.soup.find_all("a")]
            self._links = [link for link in links if link]
        return self._links


# ============================================================================
# PROMPT TEMPLATES
# ============================================================================

system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""


# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def messages_for_content(content):
    """
    Build messages list for Chat Completions API.
    
    Works with OpenAI and Ollama (OpenAI-compatible endpoints).
    """
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + content}
    ]


# ============================================================================
# MAIN FUNCTIONS
# ============================================================================

def summarize(url, model=OLLAMA_MODEL, verbose=False):
    """
    Summarize website using Ollama.
    
    Args:
        url: Website URL
        model: Ollama model name
        verbose: Print progress info
        
    Returns:
        Summary string
        
    Raises:
        ValueError: Scraping failed
        RuntimeError: LLM API call failed
    """
    if verbose:
        print(f"🔍 Fetching: {url}")
    
    try:
        website = WebsiteScraper(url)
        if verbose:
            print(f"Scraped {len(website.content)} chars")
            print(f"Summarizing with {model}...")
    except ValueError as e:
        raise ValueError(f"Scraping failed: {e}")
    
    try:
        response = ollama.chat.completions.create(
            model=model,
            messages=messages_for_content(website.content)
        )
        if verbose:
            print("Summary complete")
        return response.choices[0].message.content
    except Exception as e:
        raise RuntimeError(f"LLM API call failed: {e}")


def summarize_with_preset(url, preset="balanced", verbose=False):
    """
    Summarize using preset model configuration.
    
    Convenience wrapper for quick model switching.
    """
    model = AVAILABLE_MODELS.get(preset, OLLAMA_MODEL)
    if verbose:
        print(f"Using preset: {preset} ({model})")
    return summarize(url, model=model, verbose=verbose)


def display_summary(url, model=OLLAMA_MODEL, show_metadata=True, verbose=False):
    """
    Fetch and display website summary.
    
    Main entry point - handles scraping, summarization, and display.
    """
    try:
        website = WebsiteScraper(url)
        
        if show_metadata:
            print(f"**Title:** {website.title}")
            print(f"**Content length:** {len(website.content)} chars")
            print(f"**Model:** {model}")
            print("---\n")
        
        summary = summarize(url, model=model, verbose=verbose)
        display(Markdown(summary))
        
    except (ValueError, RuntimeError) as e:
        print(f"Error: {e}")


def compare_models(url, ollama_model=OLLAMA_MODEL, verbose=False):
    """
    Compare Ollama (local) vs OpenAI (frontier) summaries.
    
    Shows tradeoff: local/free/private vs cloud/paid/better quality.
    """
    website = WebsiteScraper(url)
    
    print(f"Comparing models for: {url}\n")
    print("=" * 70)
    
    # Ollama (local)
    print("\n Ollama (Local - Free, Private):")
    print(f"   Model: {ollama_model}")
    print("-" * 70)
    try:
        ollama_summary = summarize(url, model=ollama_model, verbose=verbose)
        display(Markdown(ollama_summary))
    except Exception as e:
        print(f"Error: {e}")
    
    # OpenAI (frontier)
    print("\n" + "=" * 70)
    print("\n OpenAI (Frontier - Paid, Cloud):")
    print("   Model: gpt-4o-mini")
    print("-" * 70)
    try:
        load_dotenv()
        openai_key = os.getenv('OPENAI_API_KEY')
        if openai_key:
            openai_client = OpenAI()
            response = openai_client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages_for_content(website.content)
            )
            display(Markdown(response.choices[0].message.content))
        else:
            print("(OpenAI API key not found - skipping)")
    except Exception as e:
        print(f" Error: {e}")
    
    print("\n" + "=" * 70)
    print("\n Tradeoff: Ollama = free/private/local, OpenAI = better quality/paid/cloud")


# ============================================================================
# USAGE
# ============================================================================

# display_summary("https://ollama.com/")
# display_summary("https://ollama.com/", verbose=True)
# display_summary("https://ollama.com/", model=AVAILABLE_MODELS["fast"])
# compare_models("https://ollama.com/")

**Title:** Ollama
**Content length:** 386 chars
**Model:** llama3.2
---



A sleek and modern AI-powered chatbot platform. Because the world was missing that.
Here's a summary of their "newsworthy" stuff:

Apparently, [Ollama] is now shipping AI-powered cloud models on major operating systems. How exciting? Additionally, it looks like they've got a blog where they'll be sharing their thoughts and expertise because why not.

Oh, and some random updates include the availability of new features: it's now possible to chat and build with open models. You know, for those who enjoy playing with AI toys.

That's it from Ollama - unless you count the obligatory copyright notices to ensure we all remember the corporate magic.